Hi all! This notebook is my compiled and tidied version of basic web scraping with python using Beautiful Soup.

The following is my experimental subject

https://www.linkedin.com/learning/search?trk=homepage-basic_intent-module-learning&sortBy=RELEVANCE&entityType=COURSE

In [ ]:
# importing the libraries
import requests # to allow http requests
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from bs4 import BeautifulSoup
import os
import urllib.request as request

In [ ]:
# begin with the url of the list of courses
url = 'https://www.linkedin.com/learning/search?trk=homepage-basic_intent-module-learning&sortBy=RELEVANCE&entityType=COURSE'
# req = requests.get(url).text
# soup = BeautifulSoup(req)
# courses = soup.find_all('li',{'class':'results-list__item'})



#ssr配置代理
proxies={'http': 'http://127.0.0.1:4780', 'https': 'http://127.0.0.1:4780'}
url='https://www.linkedin.com/learning/search?trk=homepage-basic_intent-module-learning&sortBy=RELEVANCE&entityType=COURSE'
r = requests.get(url,proxies=proxies)


soup = BeautifulSoup(r)
courses = soup.find_all('li',{'class':'results-list__item'})

In [ ]:
# empty arrays for each course seen on that list
urlList = []
durationList = []
nameList = []
byList = []
viewCountList = []
releaseDateList = []

# loop through the courses to extract information
for course in courses:
    url = course.find(href=True)
    urlList.append(url['href'])
    name = course.find('h3',{'class':'base-search-card__title'}).text.strip()
    nameList.append(name)
    by = course.find('h4',{'class':'base-search-card__subtitle'}).text.strip()
    byList.append(by)
    duration = course.find('div',{'class':'search-entity-media__duration'}).text.strip()
    durationList.append(duration)
    metadataItem = course.find_all('span')
    if "Released" in metadataItem[0].text:
        viewCountList.append(0)
        releaseDateList.append(metadataItem[0].text)
    else:
        viewCountList.append(metadataItem[0].text)
        releaseDateList.append(metadataItem[1].text)

In [ ]:
# form our dataframe from the obtained list
df = pd.DataFrame(list(zip(nameList,urlList,durationList,byList,viewCountList,releaseDateList)),
                 columns = ['course name','url','duration','by','viewer count','release date'])
df

In [ ]:
# empty arrays for items we could find from each course's url
likesList = []
skillLevelList = []
ratingList = []
ratingMaxList = []

# loop through each url to obtain information found on each webpage. If information not found, append None
for i in df['url']:
    req = requests.get(i).text
    soup = BeautifulSoup(req)
    temp = soup.find_all('span',{'class':'top-card__headline-row-item'})
    likes = None
    skill = None
    for i in temp:
        if "Liked" in i.text:
            likes = i.text
        if "Skill" in i.text:
            skill = i.text
    likesList.append(likes)
    skillLevelList.append(skill)
    rating = soup.find('span',{'class':'ratings-summary__rating-average'})
    if rating is not None:
        ratingList.append(rating.text)
    else:
        ratingList.append(None)
    ratingMax = soup.find('span',{'class':'ratings-summary__rating-max'})
    if rating is not None:
        ratingMaxList.append(ratingMax.text)
    else:
        ratingMaxList.append(None)

In [ ]:
# form dataframe from obtained list
tempdf = pd.DataFrame(list(zip(likesList,skillLevelList,ratingList,ratingMaxList)),
                     columns = ['likes','skill level','rating','rating max'])

# combine our two dataframe
df = pd.concat([df,tempdf],axis = 1)
df

In [ ]:
# save dataframe to csv file.
os.chdir(r'/kaggle/working')

df.to_csv(r'webScrapexp.csv', index = False)

This is the end of the page. Thanks for viewing!

Link to the two more detailed notebook including my thought processes:

https://www.kaggle.com/code/eugenetanake/basic-web-scraping-with-python-pt-1

https://www.kaggle.com/code/eugenetanake/basic-web-scraping-with-python-pt-2